In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [2]:
raw_text = open('./Winston_Churchil.txt').read()
raw_text = raw_text.lower()


In [3]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))


In [4]:
chars

['\n',
 ' ',
 '!',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '“',
 '”']

In [5]:
len(chars)

59

In [6]:
len(raw_text)

956204

In [7]:
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '#': 3,
 '$': 4,
 '%': 5,
 '&': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '*': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '?': 27,
 '@': 28,
 '[': 29,
 ']': 30,
 'a': 31,
 'b': 32,
 'c': 33,
 'd': 34,
 'e': 35,
 'f': 36,
 'g': 37,
 'h': 38,
 'i': 39,
 'j': 40,
 'k': 41,
 'l': 42,
 'm': 43,
 'n': 44,
 'o': 45,
 'p': 46,
 'q': 47,
 'r': 48,
 's': 49,
 't': 50,
 'u': 51,
 'v': 52,
 'w': 53,
 'x': 54,
 'y': 55,
 'z': 56,
 '“': 57,
 '”': 58}

In [8]:
int_to_char

{0: '\n',
 1: ' ',
 2: '!',
 3: '#',
 4: '$',
 5: '%',
 6: '&',
 7: "'",
 8: '(',
 9: ')',
 10: '*',
 11: ',',
 12: '-',
 13: '.',
 14: '/',
 15: '0',
 16: '1',
 17: '2',
 18: '3',
 19: '4',
 20: '5',
 21: '6',
 22: '7',
 23: '8',
 24: '9',
 25: ':',
 26: ';',
 27: '?',
 28: '@',
 29: '[',
 30: ']',
 31: 'a',
 32: 'b',
 33: 'c',
 34: 'd',
 35: 'e',
 36: 'f',
 37: 'g',
 38: 'h',
 39: 'i',
 40: 'j',
 41: 'k',
 42: 'l',
 43: 'm',
 44: 'n',
 45: 'o',
 46: 'p',
 47: 'q',
 48: 'r',
 49: 's',
 50: 't',
 51: 'u',
 52: 'v',
 53: 'w',
 54: 'x',
 55: 'y',
 56: 'z',
 57: '“',
 58: '”'}

In [9]:
seq_length = 100 #设置前100个是训练，用来预测第101个
x = []
y = []
for i in range(0, len(raw_text) - seq_length):
    given = raw_text[i:i + seq_length]
    predict = raw_text[i + seq_length]
    x.append([char_to_int[char] for char in given])
    y.append(char_to_int[predict])


In [10]:
print(x[:1])


[[50, 38, 35, 1, 46, 48, 45, 40, 35, 33, 50, 1, 37, 51, 50, 35, 44, 32, 35, 48, 37, 1, 35, 32, 45, 45, 41, 1, 45, 36, 1, 50, 38, 35, 1, 33, 48, 39, 49, 39, 49, 11, 1, 33, 45, 43, 46, 42, 35, 50, 35, 11, 1, 32, 55, 1, 53, 39, 44, 49, 50, 45, 44, 1, 33, 38, 51, 48, 33, 38, 39, 42, 42, 0, 29, 31, 51, 50, 38, 45, 48, 1, 39, 49, 1, 50, 38, 35, 1, 31, 43, 35, 48, 39, 33, 31, 44, 1, 53, 39]]


In [11]:
len(x)

956104

In [12]:
print(y[:1])

[44]


In [13]:
raw_text[:100]

'the project gutenberg ebook of the crisis, complete, by winston churchill\n[author is the american wi'

In [14]:
raw_text[:101] # 看第101个是n 序号是44

'the project gutenberg ebook of the crisis, complete, by winston churchill\n[author is the american win'

In [15]:
n_patterns = len(x)
n_vocab = len(chars)



In [16]:
n_patterns

956104

In [17]:
n_vocab

59

In [18]:
# 把x变成LSTM需要的样子
x = numpy.reshape(x, (-1, seq_length, 1))

## 关于变成模型需要的样子这一点，和之前的图片处理那里，喂给模型的数据是[batch_size,宽，高，通道数] ，原来就是用的reshape呀

In [19]:
raw_text[-2]

's'

In [20]:
raw_text[-3]

'k'

In [21]:
x

array([[[50],
        [38],
        [35],
        ...,
        [ 1],
        [53],
        [39]],

       [[38],
        [35],
        [ 1],
        ...,
        [53],
        [39],
        [44]],

       [[35],
        [ 1],
        [46],
        ...,
        [39],
        [44],
        [49]],

       ...,

       [[ 1],
        [50],
        [45],
        ...,
        [32],
        [45],
        [45]],

       [[50],
        [45],
        [ 1],
        ...,
        [45],
        [45],
        [41]],

       [[45],
        [ 1],
        [38],
        ...,
        [45],
        [41],
        [49]]])

In [22]:
x.shape # 我一直在想，这里为什么是(956104, 100, 1) 他们相乘也不等于样本数啊，

(956104, 100, 1)

In [23]:
# 简单normal到0-1之间
x = x / float(n_vocab)

In [24]:
# output变成one-hot
y = np_utils.to_categorical(y)


In [25]:
print(x[0])
print(y[0])


[[0.84745763]
 [0.6440678 ]
 [0.59322034]
 [0.01694915]
 [0.77966102]
 [0.81355932]
 [0.76271186]
 [0.6779661 ]
 [0.59322034]
 [0.55932203]
 [0.84745763]
 [0.01694915]
 [0.62711864]
 [0.86440678]
 [0.84745763]
 [0.59322034]
 [0.74576271]
 [0.54237288]
 [0.59322034]
 [0.81355932]
 [0.62711864]
 [0.01694915]
 [0.59322034]
 [0.54237288]
 [0.76271186]
 [0.76271186]
 [0.69491525]
 [0.01694915]
 [0.76271186]
 [0.61016949]
 [0.01694915]
 [0.84745763]
 [0.6440678 ]
 [0.59322034]
 [0.01694915]
 [0.55932203]
 [0.81355932]
 [0.66101695]
 [0.83050847]
 [0.66101695]
 [0.83050847]
 [0.18644068]
 [0.01694915]
 [0.55932203]
 [0.76271186]
 [0.72881356]
 [0.77966102]
 [0.71186441]
 [0.59322034]
 [0.84745763]
 [0.59322034]
 [0.18644068]
 [0.01694915]
 [0.54237288]
 [0.93220339]
 [0.01694915]
 [0.89830508]
 [0.66101695]
 [0.74576271]
 [0.83050847]
 [0.84745763]
 [0.76271186]
 [0.74576271]
 [0.01694915]
 [0.55932203]
 [0.6440678 ]
 [0.86440678]
 [0.81355932]
 [0.55932203]
 [0.6440678 ]
 [0.66101695]
 [0.71

In [26]:
model = Sequential()
model.add(LSTM(128, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


NotImplementedError: Cannot convert a symbolic Tensor (lstm/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported